In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/589_miniproject/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/589_miniproject


In [ ]:
import os
import pandas as pd
import numpy as np
import numpy as np 
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier


In [ ]:
train_data = pd.read_csv('train.csv', encoding='utf-8')
train_data = train_data.drop(['Id'],axis=1)
xtrain = train_data.iloc[:,:-1]
ytrain = train_data.pop('Pawpularity')
x_train, x_test, y_train, y_test = train_test_split(xtrain, ytrain, test_size=0.15, random_state=1)

In [ ]:
k_fold = KFold(n_splits=6)
train=train_data
best_model=None
rmse_for_valid_data = list()
best_rmse = np.inf
for i, j in k_fold.split(train_data):
    X_train= train.iloc[i]
    X_valid = train.iloc[j]
    y_train = ytrain.iloc[i]
    y_valid = ytrain.iloc[j]
    est = [100, 110, 120, 125, 150, 175, 180, 200]
    depth = [5, 10]
    for k in est:
      for l in depth:
        dt = DecisionTreeClassifier(max_depth = l, random_state=0)
        lr = AdaBoostClassifier(base_estimator = dt, n_estimators=k, learning_rate=0.1, random_state=0)
        lr.fit(X_train, y_train)
        y_pred_valid = lr.predict(X_valid)
        rmse = np.sqrt(mean_squared_error(y_valid, y_pred_valid))
        rmse_for_valid_data.append(rmse)
        if best_rmse > rmse:
          best_rmse = rmse
          best_model = lr

print(f'Best model : {best_model} with rmse : {best_rmse}')


Best model : AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5,
                                                         random_state=0),
                   learning_rate=0.1, n_estimators=100, random_state=0) with rmse : 23.06837924660888


In [11]:
y_pred_test = best_model.predict(x_test)
print(f'RMSE for predicted data : {np.sqrt(mean_squared_error(y_test, y_pred_test))}')
test = pd.read_csv('test.csv')
x_test_soln = test.drop(['Id'],axis=1) 
test['Pawpularity'] = best_model.predict(x_test_soln) 
sol = test[['Id','Pawpularity']]
sol.to_csv("ab_sol.csv", index=False)
print(best_model.predict(x_test_soln))

RMSE for predicted data : 22.27848440802232
[37 28 40 54 22 55 18 27]
